In [75]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [76]:
def imagepath_to_captcha(imagepath):
    return imagepath.split("_")[1].split(".")[0]

def load_dataset(filepath):
    filenames = os.listdir(filepath)
    x, y = [], []
    for file in filenames:
        imagepath = filepath + "/" + file
        image = tf.io.read_file(imagepath)
        image = tf.image.decode_image(image)
        pixels = tf.image.convert_image_dtype(image, tf.float32)
        pixels = tf.reshape(pixels, [-1, 3])
        # To show images: 
        # plt.imshow(tf.reshape(pixels, image.shape))
        # plt.title('Image as Pixels')
        # plt.show()
        x.append(pixels)
        y.append(imagepath_to_captcha(file))
    return (tf.convert_to_tensor(x), tf.convert_to_tensor(y))


# Read input data

In [77]:
train_x, train_y = load_dataset("captchas/train")
test_x, test_y = load_dataset("captchas/test")

print("Training input shape:", train_x.shape)
print("Training output shape:", train_y.shape)
print("Testing input shape:", test_x.shape)
print("Testing output shape:", test_x.shape)

Training input shape: (8501, 12500, 3)
Training output shape: (8501,)
Testing input shape: (1500, 12500, 3)
Testing output shape: (1500, 12500, 3)


TensorShape([12500, 3])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 54,  3], dtype=int32)>